In [1]:
import gradio as gr
import os
#import shutil
import pandas as pd
import gradio as gr
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
#import seaborn as sns
from pylab import rcParams
import numpy as np
import matplotlib.pyplot as plt
tf.get_logger().setLevel('ERROR')

#sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
#sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.preprocessing import LabelBinarizer

In [2]:
traindf = pd.read_csv('train.csv',encoding = 'utf-8')
testdf = pd.read_csv('test.csv',encoding = 'utf-8')
validdf = pd.read_csv('validation.csv',encoding = 'utf-8')

In [3]:
traindf.head()

,text,intent
0,can you help me to create an account,CreateAccount
1,User call to check claims status,CheckStatus
2,Caller had questions about the provider relief...,CallForInformation
3,have their EHB account enabled and password reset,UnlockAccount
4,I need assistance with creating an EHB account,CreateEHBAccount


In [4]:
trainfeatures = traindf.copy()
trainlabels = trainfeatures.pop('intent')

In [5]:
trainfeatures=trainfeatures.values

In [6]:
binarizer = LabelBinarizer()
trainlabels = binarizer.fit_transform(trainlabels.values)

In [7]:
trainlabels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [8]:
binarizer.classes_

array(['AccessAccount', 'AccessGrant', 'AddGrant', 'AddUser',
       'ApplyForGrant', 'ApplyForProgram', 'AskAboutLoanRepaymentProgram',
       'AskAboutStatus', 'CallForAssistance', 'CallForInformation',
       'ChangePassword', 'CheckStatus', 'CreateAccount',
       'CreateEHBAccount', 'ExtendDeadline\t', 'ExtendDueDate',
       'FileComplaint', 'FindHealthCenter', 'GetAccess', 'GetLoan',
       'GetUpdate', 'GiveAccess', 'LogIntoEHB', 'MakeChanges',
       'RegisterAsPD', 'RegisterGrant', 'RemoveUser',
       'ReportUnableToLogin', 'RequestExtension', 'ResetPassword',
       'SpeakWithSomeone', 'SubmitPriorApprovalRequest', 'UnlockAccount'],
      dtype='<U28')

In [9]:
abc = binarizer.classes_.tolist()

In [33]:
#xyz = abc.tolist()

In [10]:
abc

['AccessAccount',
 'AccessGrant',
 'AddGrant',
 'AddUser',
 'ApplyForGrant',
 'ApplyForProgram',
 'AskAboutLoanRepaymentProgram',
 'AskAboutStatus',
 'CallForAssistance',
 'CallForInformation',
 'ChangePassword',
 'CheckStatus',
 'CreateAccount',
 'CreateEHBAccount',
 'ExtendDeadline\t',
 'ExtendDueDate',
 'FileComplaint',
 'FindHealthCenter',
 'GetAccess',
 'GetLoan',
 'GetUpdate',
 'GiveAccess',
 'LogIntoEHB',
 'MakeChanges',
 'RegisterAsPD',
 'RegisterGrant',
 'RemoveUser',
 'ReportUnableToLogin',
 'RequestExtension',
 'ResetPassword',
 'SpeakWithSomeone',
 'SubmitPriorApprovalRequest',
 'UnlockAccount']

In [12]:
for each_ele in abc:
    print(each_ele)

AccessAccount
AccessGrant
AddGrant
AddUser
ApplyForGrant
ApplyForProgram
AskAboutLoanRepaymentProgram
AskAboutStatus
CallForAssistance
CallForInformation
ChangePassword
CheckStatus
CreateAccount
CreateEHBAccount
ExtendDeadline	
ExtendDueDate
FileComplaint
FindHealthCenter
GetAccess
GetLoan
GetUpdate
GiveAccess
LogIntoEHB
MakeChanges
RegisterAsPD
RegisterGrant
RemoveUser
ReportUnableToLogin
RequestExtension
ResetPassword
SpeakWithSomeone
SubmitPriorApprovalRequest
UnlockAccount


In [13]:
abc[1]

'AccessGrant'

In [11]:
type(binarizer.classes_)

numpy.ndarray

In [11]:
testfeatures=testdf.copy()
testlabels=testfeatures.pop("intent")
validfeatures=validdf.copy()
validlabels=validfeatures.pop("intent")

testfeatures=testfeatures.values
validfeatures=validfeatures.values

testlabels=binarizer.transform(testlabels.values)
validlabels=binarizer.transform(validlabels.values)

In [12]:
binarizer.classes_

array(['AccessAccount', 'AccessGrant', 'AddGrant', 'AddUser',
       'ApplyForGrant', 'ApplyForProgram', 'AskAboutLoanRepaymentProgram',
       'AskAboutStatus', 'CallForAssistance', 'CallForInformation',
       'ChangePassword', 'CheckStatus', 'CreateAccount',
       'CreateEHBAccount', 'ExtendDeadline\t', 'ExtendDueDate',
       'FileComplaint', 'FindHealthCenter', 'GetAccess', 'GetLoan',
       'GetUpdate', 'GiveAccess', 'LogIntoEHB', 'MakeChanges',
       'RegisterAsPD', 'RegisterGrant', 'RemoveUser',
       'ReportUnableToLogin', 'RequestExtension', 'ResetPassword',
       'SpeakWithSomeone', 'SubmitPriorApprovalRequest', 'UnlockAccount'],
      dtype='<U28')

In [13]:
loaded_model = keras.models.load_model("ModelSave/BERTImprovedPerf")

In [20]:
def print_my_examples_loaded(description,loaded_model):
    result = tf.nn.softmax(loaded_model(tf.constant([description])))
    print(result)
    
    confidence_score = tf.reduce_max(result, axis=-1).numpy()
    print(confidence_score[0])
    if confidence_score[0] < 0.15:
        return "Unidentified Intent"
    else:
        intents=binarizer.inverse_transform(result.numpy())
        print(result.numpy()[0])
        return intents[0]

In [12]:
# def print_my_examples_loaded(description,loaded_model):
#     result = tf.nn.softmax(loaded_model(tf.constant([description])))
#     print(result.numpy())
#     print(type(result.numpy()))
#     confidence_score = tf.reduce_max(result, axis=-1).numpy()
#     print(confidence_score[0])
#     if confidence_score[0] < 0.15:
#         return "Unidentified Intent"
#     else:
#         intents=binarizer.inverse_transform(result.numpy())
#         print(result.numpy()[0])
#         return intents[0]

In [16]:
desc_sample = "I need to speak to someone regarding my password reset."

In [17]:
result1 = tf.nn.softmax(loaded_model(tf.constant(desc_sample)))

ValueError: Could not find matching function to call loaded from the SavedModel. Got:
  Positional arguments (2 total):
    * Tensor("inputs:0", shape=(), dtype=string)
    * False
  Keyword arguments: {}

Expected these arguments to match one of the following 2 option(s):

Option 1:
  Positional arguments (2 total):
    * TensorSpec(shape=(None,), dtype=tf.string, name='inputs')
    * False
  Keyword arguments: {}

Option 2:
  Positional arguments (2 total):
    * TensorSpec(shape=(None,), dtype=tf.string, name='inputs')
    * True
  Keyword arguments: {}

In [ ]:
result

In [21]:
ab = tf.constant(desc_sample)

In [22]:
ab

<tf.Tensor: shape=(), dtype=string, numpy=b'I need to speak to someone regarding my password reset.'>

In [20]:
result1

NameError: name 'result1' is not defined

In [155]:
result1[0][1]

<tf.Tensor: shape=(), dtype=float32, numpy=0.026304888>

In [167]:
softmax_output = [result1.numpy()[0].tolist()]
shape = result1.shape[1]

In [168]:
softmax_output

[[0.027228279039263725,
  0.026304887607693672,
  0.08228589594364166,
  0.0036031268537044525,
  0.44955044984817505,
  0.018399879336357117,
  0.006225119344890118,
  0.0036713078152388334,
  0.00781845860183239,
  0.007059742230921984,
  0.017932118847966194,
  0.006314810831099749,
  0.002523731905966997,
  0.021285846829414368,
  0.007056572008877993,
  0.004764123819768429,
  0.015213764272630215,
  0.00854277890175581,
  0.010263376869261265,
  0.002781044226139784,
  0.0015029258793219924,
  0.02185211516916752,
  0.008831911720335484,
  0.010866662487387657,
  0.012651135213673115,
  0.07413958013057709,
  0.014079182408750057,
  0.06972690671682358,
  0.010838625021278858,
  0.017773078754544258,
  0.005516902077943087,
  0.007846600376069546,
  0.015549001283943653]]

In [164]:
softmax_output.tolist()

[0.027228279039263725,
 0.026304887607693672,
 0.08228589594364166,
 0.0036031268537044525,
 0.44955044984817505,
 0.018399879336357117,
 0.006225119344890118,
 0.0036713078152388334,
 0.00781845860183239,
 0.007059742230921984,
 0.017932118847966194,
 0.006314810831099749,
 0.002523731905966997,
 0.021285846829414368,
 0.007056572008877993,
 0.004764123819768429,
 0.015213764272630215,
 0.00854277890175581,
 0.010263376869261265,
 0.002781044226139784,
 0.0015029258793219924,
 0.02185211516916752,
 0.008831911720335484,
 0.010866662487387657,
 0.012651135213673115,
 0.07413958013057709,
 0.014079182408750057,
 0.06972690671682358,
 0.010838625021278858,
 0.017773078754544258,
 0.005516902077943087,
 0.007846600376069546,
 0.015549001283943653]

In [131]:
final_array = np.array(softmax_output,[shape])

TypeError: Field elements must be 2- or 3-tuples, got '33'

In [122]:
softmax_output

array([0.02722828, 0.02630489, 0.0822859 , 0.00360313, 0.44955045,
       0.01839988, 0.00622512, 0.00367131, 0.00781846, 0.00705974,
       0.01793212, 0.00631481, 0.00252373, 0.02128585, 0.00705657,
       0.00476412, 0.01521376, 0.00854278, 0.01026338, 0.00278104,
       0.00150293, 0.02185212, 0.00883191, 0.01086666, 0.01265114,
       0.07413958, 0.01407918, 0.06972691, 0.01083863, 0.01777308,
       0.0055169 , 0.0078466 , 0.015549  ], dtype=float32)

In [119]:
type(softmax_output)

numpy.ndarray

array([0.02722828, 0.02630489, 0.0822859 , 0.00360313, 0.44955045,
       0.01839988, 0.00622512, 0.00367131, 0.00781846, 0.00705974,
       0.01793212, 0.00631481, 0.00252373, 0.02128585, 0.00705657,
       0.00476412, 0.01521376, 0.00854278, 0.01026338, 0.00278104,
       0.00150293, 0.02185212, 0.00883191, 0.01086666, 0.01265114,
       0.07413958, 0.01407918, 0.06972691, 0.01083863, 0.01777308,
       0.0055169 , 0.0078466 , 0.015549  ], dtype=float32)

In [105]:
softmax_output = np.append(softmax_output,33)

In [106]:
softmax_output

array([2.72282790e-02, 2.63048876e-02, 8.22858959e-02, 3.60312685e-03,
       4.49550450e-01, 1.83998793e-02, 6.22511934e-03, 3.67130782e-03,
       7.81845860e-03, 7.05974223e-03, 1.79321188e-02, 6.31481083e-03,
       2.52373191e-03, 2.12858468e-02, 7.05657201e-03, 4.76412382e-03,
       1.52137643e-02, 8.54277890e-03, 1.02633769e-02, 2.78104423e-03,
       1.50292588e-03, 2.18521152e-02, 8.83191172e-03, 1.08666625e-02,
       1.26511352e-02, 7.41395801e-02, 1.40791824e-02, 6.97269067e-02,
       1.08386250e-02, 1.77730788e-02, 5.51690208e-03, 7.84660038e-03,
       1.55490013e-02, 3.30000000e+01])

In [92]:
result1.shape[1]

33

In [72]:
result1

<tf.Tensor: shape=(1, 33), dtype=float32, numpy=
array([[0.02722828, 0.02630489, 0.0822859 , 0.00360313, 0.44955045,
        0.01839988, 0.00622512, 0.00367131, 0.00781846, 0.00705974,
        0.01793212, 0.00631481, 0.00252373, 0.02128585, 0.00705657,
        0.00476412, 0.01521376, 0.00854278, 0.01026338, 0.00278104,
        0.00150293, 0.02185212, 0.00883191, 0.01086666, 0.01265114,
        0.07413958, 0.01407918, 0.06972691, 0.01083863, 0.01777308,
        0.0055169 , 0.0078466 , 0.015549  ]], dtype=float32)>

In [21]:
#description = "Not able to apply grant"
description = "I need to speak to someone regarding my password reset."

In [61]:
res = tf.Tensor([[0.02722828 0.02630489 0.0822859  0.00360313 0.44955045 0.01839988
  0.00622512 0.00367131 0.00781846 0.00705974 0.01793212 0.00631481
  0.00252373 0.02128585 0.00705657 0.00476412 0.01521376 0.00854278
  0.01026338 0.00278104 0.00150293 0.02185212 0.00883191 0.01086666
  0.01265114 0.07413958 0.01407918 0.06972691 0.01083863 0.01777308
  0.0055169  0.0078466  0.015549  ]], shape=(1, 33), dtype=float32)

SyntaxError: invalid syntax (3890858306.py, line 2)

In [22]:
print_my_examples_loaded(description,loaded_model)

tf.Tensor(
[[0.00811819 0.02590993 0.01567079 0.00872281 0.00756187 0.00760941
  0.00794445 0.00752236 0.00856809 0.00177778 0.02388663 0.00747161
  0.00313875 0.01650795 0.0067107  0.00554407 0.00568805 0.00365409
  0.02002718 0.00522056 0.00836416 0.01523894 0.00470102 0.02018244
  0.02382814 0.01478851 0.03907108 0.00429643 0.0052806  0.03564246
  0.6107043  0.00105723 0.01958942]], shape=(1, 33), dtype=float32)
0.6107043
[0.00811819 0.02590993 0.01567079 0.00872281 0.00756187 0.00760941
 0.00794445 0.00752236 0.00856809 0.00177778 0.02388663 0.00747161
 0.00313875 0.01650795 0.0067107  0.00554407 0.00568805 0.00365409
 0.02002718 0.00522056 0.00836416 0.01523894 0.00470102 0.02018244
 0.02382814 0.01478851 0.03907108 0.00429643 0.0052806  0.03564246
 0.6107043  0.00105723 0.01958942]


'SpeakWithSomeone'

In [60]:
test =[{'class': 'SpeakWithSomeone', 'score': 0.7444775}, {'class': 'AccessGrant', 'score': 0.029643694}]
test[0]


{'class': 'SpeakWithSomeone', 'score': 0.7444775}

In [23]:
def intent_values(description):
    intent = print_my_examples_loaded(description,loaded_model)
    return intent


In [24]:
demo = gr.Interface(fn=intent_values, inputs="text", outputs="text")
demo.launch(share = True) 

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://c893c50a29f0b5a0.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x1a3016e6100>,
 'http://127.0.0.1:7861/',
 'https://c893c50a29f0b5a0.gradio.app')

tf.Tensor(
[[1.0034028e-02 1.5118038e-02 3.1475404e-03 2.4593021e-03 6.7692017e-03
  1.5484176e-03 3.4941135e-03 1.5910531e-03 8.6719440e-03 1.0598056e-03
  7.0653604e-03 9.5054014e-03 6.7729149e-03 1.6988446e-03 4.5613656e-03
  5.8626346e-03 2.2340347e-03 3.5694854e-03 2.7527949e-03 1.4305869e-02
  3.0622538e-03 4.7329869e-03 1.8496741e-02 5.4623983e-03 2.3856307e-03
  2.7378376e-03 1.8784322e-03 1.2193403e-02 3.4219952e-04 8.1034088e-01
  1.6512414e-03 7.8917919e-03 1.6602162e-02]], shape=(1, 33), dtype=float32)
0.8103409
[1.0034028e-02 1.5118038e-02 3.1475404e-03 2.4593021e-03 6.7692017e-03
 1.5484176e-03 3.4941135e-03 1.5910531e-03 8.6719440e-03 1.0598056e-03
 7.0653604e-03 9.5054014e-03 6.7729149e-03 1.6988446e-03 4.5613656e-03
 5.8626346e-03 2.2340347e-03 3.5694854e-03 2.7527949e-03 1.4305869e-02
 3.0622538e-03 4.7329869e-03 1.8496741e-02 5.4623983e-03 2.3856307e-03
 2.7378376e-03 1.8784322e-03 1.2193403e-02 3.4219952e-04 8.1034088e-01
 1.6512414e-03 7.8917919e-03 1.6602162e-02]
